# PEFT Tour: LoRA vs Prefix vs Prompt (Side‑by‑Side)
Compare parameter counts and basic setup across methods on a causal LM.

In [ ]:
!pip -q install -U transformers peft accelerate datasets


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, PrefixTuningConfig, PromptTuningConfig, TaskType

base = "distilgpt2"
tok = AutoTokenizer.from_pretrained(base)
tok.pad_token = tok.eos_token
model_base = AutoModelForCausalLM.from_pretrained(base)
print("Base params:", sum(p.numel() for p in model_base.parameters())/1e6, "M")


In [ ]:
# LoRA
lora_cfg = LoraConfig(r=16, lora_alpha=32, target_modules=["c_attn","c_proj"], lora_dropout=0.05, task_type=TaskType.CAUSAL_LM)
lora_model = get_peft_model(model_base, lora_cfg)
print("LoRA trainable params:", sum(p.numel() for p in lora_model.parameters() if p.requires_grad)/1e6, "M")


In [ ]:
# Prefix
pre_cfg = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
pre_model = get_peft_model(AutoModelForCausalLM.from_pretrained(base), pre_cfg)
print("Prefix trainable params:", sum(p.numel() for p in pre_model.parameters() if p.requires_grad)/1e6, "M")


In [ ]:
# Prompt
pro_cfg = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=20)
pro_model = get_peft_model(AutoModelForCausalLM.from_pretrained(base), pro_cfg)
print("Prompt trainable params:", sum(p.numel() for p in pro_model.parameters() if p.requires_grad)/1e6, "M")


**Exercise:** Plug any of these PEFT configs into your SFT trainer and compare convergence vs full‑fine‑tune.